In [1]:
# !pip install skicit-learn

import sklearn
import pandas as pd
import json
import numpy as np

In [2]:
with open('fights_df.json', 'r') as file:
    fights = json.load(file)
    df = pd.DataFrame(fights)

with open('fight_results.json', 'r') as file:
    y = json.load(file)

print(len(df), len(df.columns))
print(len(y))
print(df.head())

1310 133
1310
   Fight_info_division  Fight_info_n_rounds  Fighter_A_basic_stats_Height  \
0                    5                    5                            70   
1                    3                    3                            73   
2                    4                    3                            69   
3                    5                    5                            71   
4                    3                    3                            67   

   Fighter_A_basic_stats_Reach  Fighter_A_basic_stats_Stance_0  \
0                           71                               0   
1                           74                               1   
2                           68                               0   
3                           72                               0   
4                           70                               0   

   Fighter_A_basic_stats_Stance_1  Fighter_A_basic_stats_Age  \
0                               1                      38.83  

In [3]:
import numpy as np
from feature_engineering import engineer_features

new_df = engineer_features(df)

with open('X.json', "w") as file:
    json.dump(new_df.to_dict(orient="records"), file)

with open('X.json', "r") as file:
    X = json.load(file)
    X = pd.DataFrame.from_dict(X)

print(X.head())
print(X.columns)
print(X.iloc[0])

   Fight_kd_diff  Fight_striking_ovr_acc_diff  Fight_mean_striking_diff  \
0      -0.639756                    -0.064304                 -2.245657   
1      -0.421790                     0.052886                 -0.543298   
2       0.002363                     0.098326                  0.958598   
3      -1.022978                    -0.241565                 -1.123686   
4      -1.081639                    -0.087471                 -0.359971   

   Fight_median_striking_diff  Fight_strikes_head_diff  \
0                   -9.500000                -0.080887   
1                   -3.560901                -0.471259   
2                   15.333333                -0.177444   
3                  -12.833333                 0.096647   
4                    2.863180                -0.226605   

   Fight_strikes_body_diff  Fight_strikes_leg_diff  \
0                 0.349110               -0.274122   
1                 0.405926                0.399433   
2                -0.062384            

In [4]:
with open('fight_results.json', 'r') as file:
    y = json.load(file)
    y_df = pd.DataFrame.from_dict(y)

y_df['winner'] = y_df['winner'].replace({'A': 0, 'B': 1, 'Draw/NC': 2})
print(y_df.head())

                 id  winner
0  a81ad236a2f221f2       1
1  fe2acfdaaaec3dba       1
2  8ba31cf0b3164b72       0
3  b873c116e39c9920       1
4  891b2faf1bff38ee       1


C:\Users\Rober\AppData\Local\Temp\ipykernel_14000\2140858880.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_df['winner'] = y_df['winner'].replace({'A': 0, 'B': 1, 'Draw/NC': 2})


In [5]:
import numpy as np

y = y_df['winner']

# get indices where y == 2 (draw/no contest)
# draw or no contest are very rare, difficult to predict
indices_to_remove = np.where(y == 2)[0] # from test too?

X_filtered = X.drop(index=indices_to_remove).reset_index(drop=True)
y_filtered = y.drop(index=indices_to_remove).reset_index(drop=True)

print(X_filtered.shape, y_filtered.shape)

(1285, 50) (1285,)


**BASE/NAIVE MODEL: PREDICTING ON THE FAVOURITE. PROBLEM => ACCESSING ODDS**

In [6]:
train_set_size = int(len(X_filtered)*0.8)
val_set_size = int(len(y_filtered)*0.1)
# X = X_std[:,:12]
X_train = X_filtered[-train_set_size:]
X_val = X_filtered[val_set_size:-train_set_size]
X_test = X_filtered[:-train_set_size-val_set_size]
y_train = y_filtered[-train_set_size:]
y_val = y_filtered[val_set_size:-train_set_size]
y_test = y_filtered[:-train_set_size-val_set_size]

print(X_filtered.shape, y_filtered.shape)

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(1285, 50) (1285,)
(1028, 50) (1028,) (129, 50) (129,) (129, 50) (129,)


**ML models**

**Check train results**

In [7]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train_std, y_train)
print(gpr.score(X_train_std, y_train))
print(f"Val score for Gaussian: {gpr.score(X_val_std, y_val)}")

0.1258943782816776
Val score for Gaussian: 0.11081141559154872


In [8]:
# !pip install xgboost

import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)


best_r2 = 0
best_config = None

# xgb_model.load_model("xgb_best_model.json") # i should only load the parameters actually, not the fitted model
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.003,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train_std, y_train)

# train evaluation
y_train_pred = xgb_model.predict(X_train_std)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)
print(f"Train MSE: {train_mse:.4f}, Train R2: {train_r2:.4f}")

# validation
y_val_pred = xgb_model.predict(X_val_std)
y_val_pred_adj = np.round(y_val_pred).astype(int)
acc = len(np.where(y_val_pred_adj == y_val)[0]) / len(y_val_pred_adj)
test_mse = mean_squared_error(y_val, y_val_pred)
test_r2 = r2_score(y_val, y_val_pred)
print(f"Test MSE: {test_mse:.4f}, Val R²: {test_r2:.4f}\n")

Train MSE: 0.0403, Train R2: 0.8363
Test MSE: 0.2297, Val R²: 0.0808



In [9]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

svm_reg = make_pipeline(StandardScaler(), SVR())
svm_reg.fit(X_train_std, y_train)
svm_r2_score = svm_reg.score(X_val_std, y_val)

print("R² val score for SVR:", svm_r2_score)

import xgboost as xgb

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=11,
    learning_rate=0.003,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(X_train_std, y_train)
y_pred = xgb_model.predict(X_val_std)
# xgb_val_r2 = r2_score(y_pred, y_val)
xgb_val_r2 = xgb_model.score(X_val_std, y_val)
print(f"R² val score for XGBRegressor: {xgb_val_r2}")

from sklearn.gaussian_process.kernels import RBF, Matern

# try different kernels

kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train_std, y_train)
gpr_val_r2 = gpr.score(X_val_std, y_val)
print(f"R² val score for GaussianProcessRegressor 1: {gpr_val_r2}")

kernel = Matern() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train_std, y_train)
gpr_val_r2 = gpr.score(X_val_std, y_val)
print(f"R² val score for GaussianProcessRegressor 2: {gpr_val_r2}")

kernel = RBF() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train_std, y_train)
gpr_val_r2 = gpr.score(X_val_std, y_val)
print(f"R² val score for GaussianProcessRegressor 3: {gpr_val_r2}")

kernel = RBF() + DotProduct()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train_std, y_train)
gpr_val_r2 = gpr.score(X_val_std, y_val)
print(f"R² val score for GaussianProcessRegressor 4: {gpr_val_r2}")

kernel = Matern() + RBF() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_train_std, y_train)
gpr_val_r2 = gpr.score(X_val_std, y_val)
print(f"R² val score for GaussianProcessRegressor 5: {gpr_val_r2}")
                        

from sklearn.linear_model import LinearRegression

linr = LinearRegression()
linr.fit(X_train_std, y_train)
lr_val_r2 = linr.score(X_val_std, y_val)

print(f"R² val score for LinearRegression: {lr_val_r2}")

from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(X_train_std, y_train)
rf_val_r2 = rfr.score(X_val_std, y_val)

print(f"R² val score for RandomForestRegressor: {rf_val_r2}")

from sklearn.linear_model import LogisticRegression

logr = LogisticRegression()
logr.fit(X_train_std, y_train)
log_val_r2 = logr.score(X_val_std, y_val)

print(f"Accuracy val score for LogisticRegression: {log_val_r2}")

from sklearn.linear_model import Ridge

rid = Ridge()
rid.fit(X_train_std, y_train)
ridge_val_r2 = rid.score(X_val_std, y_val)

print(f"R² val score for Ridge: {ridge_val_r2}")

from lightgbm import LGBMRegressor

lgbm = LGBMRegressor(
    n_estimators=300,
    learning_rate=0.02,
    max_depth=6,
    random_state=42,
    verbose=0
)
lgbm.fit(X_train, y_train)
lgbm_val_r2 = lgbm.score(X_val, y_val)

print(f"R² val score for LGBMRegressor: {lgbm_val_r2}")

from catboost import CatBoostRegressor

cbr = CatBoostRegressor(verbose=0)
cbr.fit(X_train_std, y_train)
cbr_val_r2 = cbr.score(X_val_std, y_val)

print(f"R² val score for CatBoostRegressor: {cbr_val_r2}")

from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(50,), max_iter=500, random_state=42)
mlp.fit(X_train_std, y_train)
mlp_val_r2 = mlp.score(X_val_std, y_val)

print(f"R² val score for MLPRegressor: {mlp_val_r2}")

R² val score for SVR: 0.011374714831225763
R² val score for XGBRegressor: 0.0808340311050415
R² val score for GaussianProcessRegressor 1: 0.11081141559154872
R² val score for GaussianProcessRegressor 2: 0.12382169609478677
R² val score for GaussianProcessRegressor 3: -0.18691038201136734
R² val score for GaussianProcessRegressor 4: -0.1891763784931566


C:\Users\Rober\miniconda3\envs\ufc\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__length_scale is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


R² val score for GaussianProcessRegressor 5: -0.1291145313628359
R² val score for LinearRegression: 0.11103765583775638
R² val score for RandomForestRegressor: 0.048496248196248426
Accuracy val score for LogisticRegression: 0.6589147286821705
R² val score for Ridge: 0.11144105212577105
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

C:\Users\Rober\miniconda3\envs\ufc\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


**Train + val: check test results of best 2 models**

In [10]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

X_trainval = np.concat((X_train, X_val))
y_trainval = np.concat((y_train, y_val))
scaler = MinMaxScaler()
X_trainval_std = scaler.fit_transform(X_trainval)
X_test_std = scaler.transform(X_test)

C:\Users\Rober\miniconda3\envs\ufc\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


In [11]:
# only uncomment the two best models

from sklearn.preprocessing import StandardScaler
# from sklearn.svm import SVR
# from sklearn.pipeline import make_pipeline

# svm_reg = make_pipeline(StandardScaler(), SVR())

# svm_reg.fit(X_trainval, y_trainval)
# svm_r2_score = svm_reg.score(X_test, y_test)

# print("R² test score for SVR:", r2_score)

# import xgboost as xgb

# xgb_model = xgb.XGBRegressor(
#     n_estimators=500,
#     max_depth=11,
#     learning_rate=0.003,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42
# )
# xgb_model.fit(X_trainval_std, y_trainval)
# y_pred = xgb_model.predict(X_test_std)
# # xgb_test_r2 = r2_score(y_pred, y_test)
# xgb_test_r2 = xgb_model.score(X_test_std, y_test)
# print(f"R² test score for XGBRegressor: {xgb_test_r2}")

from sklearn.gaussian_process.kernels import RBF, Matern

# try different kernels

# kernel = DotProduct() + WhiteKernel()
# gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_trainval_std, y_trainval)
# gpr_test_r2 = gpr.score(X_test_std, y_test)
# print(f"R² test score for GaussianProcessRegressor 1: {gpr_test_r2}")

kernel = Matern() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_trainval_std, y_trainval)
gpr_r2_test = gpr.score(X_test_std, y_test)
print(f"R² test score for GaussianProcessRegressor 2: {gpr_r2_test}")

# kernel = RBF() + WhiteKernel()
# gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_trainval_std, y_trainval)
# gpr_test_r2 = gpr.score(X_test_std, y_test)
# print(f"R² test score for GaussianProcessRegressor 3: {gpr_test_r2}")

# kernel = RBF() + DotProduct()
# gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_trainval_std, y_trainval)
# gpr_test_r2 = gpr.score(X_test_std, y_test)
# print(f"R² test score for GaussianProcessRegressor 4: {gpr_test_r2}")

# kernel = Matern() + RBF() + WhiteKernel()
# gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_trainval_std, y_trainval)
# gpr_test_r2 = gpr.score(X_test_std, y_test)
# print(f"R² test score for GaussianProcessRegressor 5: {gpr_test_r2}")
                        

# from sklearn.linear_model import LinearRegression

# linr = LinearRegression()
# linr.fit(X_trainval_std, y_trainval)
# lr_test_r2 = linr.score(X_test_std, y_test)

# print(f"R² test score for LinearRegression: {lr_test_r2}")

# from sklearn.ensemble import RandomForestRegressor

# rfr = RandomForestRegressor()
# rfr.fit(X_trainval_std, y_trainval)
# rf_test_r2 = rfr.score(X_test_std, y_test)

# print(f"R² test score for RandomForestRegressor: {rf_test_r2}")

# from sklearn.linear_model import LogisticRegression

# logr = LogisticRegression()
# logr.fit(X_trainval_std, y_trainval)
# log_test_r2 = logr.score(X_test_std, y_test)

# print(f"Accuracy test score for LogisticRegression: {log_test_r2}")

# from sklearn.linear_model import Ridge

# rid = Ridge()
# rid.fit(X_trainval_std, y_trainval)
# ridge_test_r2 = rid.score(X_test_std, y_test)

# print(f"R² test score for Ridge: {ridge_test_r2}")

# from lightgbm import LGBMRegressor

# reg = LGBMRegressor(
#     n_estimators=300,
#     learning_rate=0.02,
#     max_depth=6,
#     random_state=42,
#     verbose=0
# )
# lgbm.fit(X_trainval, y_trainval)
# lgbm_test_r2 = lgbm.score(X_test, y_test)

# print(f"R² test score for LGBMRegressor: {lgbm_test_r2}")

from catboost import CatBoostRegressor

cat = CatBoostRegressor(
    iterations=500,
    learning_rate=0.03,
    depth=4,
    l2_leaf_reg=5,
    loss_function='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=0
)
cat.fit(X_trainval_std, y_trainval)
cbr_r2_test = cat.score(X_test_std, y_test)

print(f"R² test score for CatBoostRegressor: {cbr_r2_test}")

# from sklearn.neural_network import MLPRegressor

# mlp = MLPRegressor(hidden_layer_sizes=(50,), max_iter=500, random_state=42)
# mlp.fit(X_trainval_std, y_trainval)
# mlp_test_r2 = mlp.score(X_test_std, y_test)

# print(f"R² test score for MLPRegressor: {mlp_test_r2}")

R² test score for GaussianProcessRegressor 2: 0.13716157186837008
R² test score for CatBoostRegressor: 0.1025108153700095


In [12]:
# check test results

chandler_pimblett_id = "d05cb4c4135ce402"
kattar_aljo_id = "d95f21817a56eb05"
pereira_hill_id = "e4931f3ab3bf4141"
pereira_rountree_id = "8c5e06c71f392f33"
omalley_merab_1_id = "3146e5a47a922976"
krylov_reyes_id = "b2d731415bd367df"
mitchell_silva_id = "8c540eb4afe8c43e"
bsd_moicano_id = "7ca4c91bab696b89"
borralho_craig_id = "2db0998b24c11ea0"
omalley_vera_2_id = "894c44c3d04aaf6f"
gamrot_rda_id = "8baff440b82afd2b"

row = y_df.loc[y_df['id'] == pereira_rountree_id]
print(row.index[0])
index = row.index[0]# - 240
print(X_test_std[index])

print(f"Gaussian: {round(float(gpr.predict(X_test_std[index].reshape(1,-1))[0]), 2)}")
# print(f"XGBoost: {round(float(y_test_pred[index]), 2)}")

121
[0.49934463 0.86271812 0.65138662 0.73959317 0.5743012  0.49415209
 0.46102681 0.58013459 0.42805401 0.55537175 0.60608351 0.53264816
 0.13817126 0.         0.47064547 0.50342189 0.67118644 0.7262253
 0.71783355 0.33252427 0.40837611 0.74444444 0.62099601 0.
 0.         0.63581346 0.70945461 0.7341949  0.5        0.5
 0.5        0.         0.5        1.         0.5        0.
 1.         0.5        0.         0.         0.5        0.47719298
 0.         0.63157895 0.61111111 0.         1.         0.37037037
 0.         0.5       ]
Gaussian: 0.31


**Train on all data to predict future fights**

In [13]:
full_data_scaler = StandardScaler()
X_std = full_data_scaler.fit_transform(X_filtered)


kernel = Matern() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, random_state=0).fit(X_std, y_filtered)
print(f"All train score for Gaussian 2: {gpr.score(X_std, y_filtered)}")


cat = CatBoostRegressor( # to do: optimize params
    iterations=500,
    learning_rate=0.03,
    depth=4,
    l2_leaf_reg=5,
    loss_function='RMSE',
    random_seed=42,
    early_stopping_rounds=50,
    verbose=0
)
cat.fit(X_std, y_filtered)
cbr_train_r2 = cat.score(X_std, y_filtered)
print(f"All train score for CatBoost: {cbr_train_r2}")

All train score for Gaussian 2: 0.13392564447750654
All train score for CatBoost: 0.42945953104417867


In [14]:
from datetime import datetime
from fight import get_hypothetical_fight
from fighter import get_fighter_name_from_url
from fight import encode_weight_class

def predict_hypothetical_fight(fighter_0_id, fighter_1_id, full_df, division, cutoff_date=None):
    url_0 = "http://ufcstats.com/fighter-details/"+fighter_0_id
    url_1 = "http://ufcstats.com/fighter-details/"+fighter_1_id

    fighter_0_name = get_fighter_name_from_url(url_0)
    fighter_1_name = get_fighter_name_from_url(url_1)

    if cutoff_date is None:
        cutoff_date = datetime.strptime("2025-09-20", "%Y-%m-%d").date() ## check age being calculated properly
    
    hypothetical_fight = get_hypothetical_fight(url_0, url_1, cutoff_date)
    hypothetical_fight['Fighter_A_career_stats_weight'] = hypothetical_fight['Fighter_B_career_stats_weight'] = None
    hypothetical_fight['Fight_info_division'] = division
    h_df = engineer_features(hypothetical_fight, full_df)

    
    h_df = full_data_scaler.transform(h_df)
    
    gaussian_prob = gpr.predict(h_df[0].reshape(1,-1), return_std=True)[0][0]
    catboost_prob = cat.predict(h_df[0].reshape(1,-1))[0]

    gaussian_weight = gpr_r2_test**2 / (gpr_r2_test**2 + cbr_r2_test**2)# + ridge_test_r2**2)
    catboost_weight = cbr_r2_test**2 / (gpr_r2_test**2 + cbr_r2_test**2)# + ridge_test_r2**2)

    combined_prob = round((gaussian_prob * gaussian_weight + catboost_prob * catboost_weight) * 100, 2)

    print(f"Gaussian: {gaussian_prob}")
    print(f"Cat Boost: {catboost_weight}")
    
    return combined_prob

In [15]:
def predict_hypothetical_probabilities(fighter_a_id, fighter_b_id, division, date):
    combined_prob_1 = predict_hypothetical_fight(fighter_a_id, fighter_b_id, df, division, date)
    combined_prob_2 = predict_hypothetical_fight(fighter_b_id, fighter_a_id, df, division, date)
    real_prob = combined_prob_2 / (combined_prob_1+combined_prob_2)
    fighter_a_url, fighter_b_url = "http://ufcstats.com/fighter-details/" + fighter_a_id, "http://ufcstats.com/fighter-details/" + fighter_b_id
    fighter_a_name, fighter_b_name = get_fighter_name_from_url(fighter_a_url), get_fighter_name_from_url(fighter_b_url)
    print(f"Probabilities given for {fighter_a_name} to beat {fighter_b_name}: {round(real_prob*100, 1)}%")

In [18]:
# fighter_a_name = input("Please introduce fighter A name")
# fighter_a_surname = input("Please introduce fighter A name")

fighter_a =  input("Please introduce <fighter_id> A from ufcstats.com/<fighter_id>")
fighter_b = input("Please introduce <fighter_id> B from ufcstats.com/<fighter_id>")
division = int(input("Please introduce division (1-Flyweight, 4-Lightweight, 8-Heavyweight)"))
fight_date = input(f"Please introduce the fight date in format like 'Jan 1, 2000'")
fight_datetime = datetime.strptime(fight_date.strip(), "%b %d, %Y").date()

predict_hypothetical_probabilities(fighter_a, fighter_b, division, fight_datetime)

Please introduce <fighter_id> A from ufcstats.com/<fighter_id> 7026eca45f65377b
Please introduce <fighter_id> B from ufcstats.com/<fighter_id> 7826923b47f8d72a
Please introduce division (1-Flyweight, 4-Lightweight, 8-Heavyweight) 4
Please introduce the fight date in format like 'Jan 1, 2000' Nov 1, 2025


Gaussian: 0.6379588778361054
Cat Boost: 0.35838455164048466
Gaussian: 0.2250008156025558
Cat Boost: 0.35838455164048466
Probabilities given for Jared Gordon to beat Paddy Pimblett: 26.6%
